In [1]:
import numpy as np
import pandas as pd
import time, datetime
import sys, os, os.path
import scipy, scipy.signal

from datetime import date, datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, accuracy_score, \
                            confusion_matrix, balanced_accuracy_score, \
                            classification_report
import matplotlib.pyplot as plt

# from patsy import cr
# from pprint import pprint
# from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
import NASA_plot_core as ncp

2024-06-27 09:21:30.984254: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
NASA_dir_base = "/Users/hn/Documents/01_research_data/NASA/"
meta_dir = NASA_dir_base + "0000_parameters/"
train_TS_dir_base = NASA_dir_base + "VI_TS/"

data_part_of_shapefile_dir = NASA_dir_base + "data_part_of_shapefile/"

In [4]:
out_name = "/Users/hn/Documents/01_research_data/NASA/" + "all_fields_correct_year_irr_noNass.csv"
pool = pd.read_csv(out_name)
pool.head(2)

,ID,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Shp_Lng,Shap_Ar
0,51380_WSDA_SF_2016,Seed,bluegrass seed,6,sprinkler,2003/07/01 00:00:00,2016/09/19 00:00:00,wsda,NaN,T20R35E24,Adams,Wheat,6.116621,937.384083,24753.087193
1,51381_WSDA_SF_2016,Cereal Grain,wheat,117,center pivot,2003/07/01 00:00:00,2016/09/20 00:00:00,wsda,NaN,T16R36E22,Adams,Potato,116.743101,2445.413312,472442.568255


In [5]:
meta_dir = "/Users/hn/Documents/01_research_data/NASA/parameters/"
meta = pd.read_csv(meta_dir+"evaluation_set.csv")
meta_moreThan10Acr=meta[meta.ExctAcr>10]
print (meta.shape)
print (meta_moreThan10Acr.shape)
meta.head(2)

(6340, 8)
(3539, 8)


,ID,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,100010_WSDA_SF_2017,alfalfa hay,center pivot,wsda,34,34.310305,2017/09/12,Grant
1,100204_WSDA_SF_2017,alfalfa hay,center pivot,wsda,62,61.826535,2017/08/09,Grant


In [6]:
training_set_dir = "/Users/hn/Documents/01_research_data/NASA/ML_data_Oct17/"
ground_truth_labels = pd.read_csv(training_set_dir+"groundTruth_labels_Oct17_2022.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
3160


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


In [7]:
GT_wMeta = pd.merge(ground_truth_labels, meta, on="ID", how="left")
print (GT_wMeta.shape)
GT_wMeta.head(2)

(3160, 9)


,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,99837_WSDA_SF_2017,2,barley,center pivot,wsda,73,73.250981,2017/10/10,Grant
1,114615_WSDA_SF_2017,1,barley,center pivot,wsda,31,30.993504,2017/10/10,Grant


In [8]:
GT_crops = GT_wMeta["CropTyp"].unique()
meta_crops = meta["CropTyp"].unique()

print (len(GT_crops))
print (len(meta_crops))

64
68


In [9]:
[x for x in meta_crops if not(x in GT_crops)]

['alfalfa/grass hay', 'alkali bee bed', 'caneberry', 'walnut']

In [10]:
# csv_data_parts = os.listdir(data_part_of_shapefile_dir)
# csv_data_parts = [x for x in csv_data_parts if x.endswith("csv")]
# csv_data_parts

In [11]:
# irriigated_SF_data_concatenated = pd.read_csv(data_part_of_shapefile_dir + "irriigated_SF_data_concatenated.csv")
# irriigated_SF_data_concatenated["survey_year"] = pd.to_datetime(irriigated_SF_data_concatenated["LstSrvD"], 
#                                                                 format='mixed').dt.year

# print (irriigated_SF_data_concatenated.county.unique())

# irriigated_SF_data_concatenated["SF_year"] = irriigated_SF_data_concatenated["ID"].str.split("_", expand=True)[3]
# irriigated_SF_data_concatenated["SF_year"] = irriigated_SF_data_concatenated["SF_year"].astype(int)
# irriigated_SF_data_concatenated.head(2)

In [12]:
# (irriigated_SF_data_concatenated["SF_year"] == irriigated_SF_data_concatenated["survey_year"]).sum()

# print (irriigated_SF_data_concatenated.shape)
# # limit to large fields
# irriigated_SF_data_concatenated = irriigated_SF_data_concatenated[irriigated_SF_data_concatenated.ExctAcr > 10]
# print (len(list(irriigated_SF_data_concatenated.CropTyp.unique())))
# irriigated_SF_data_concatenated.head(2)

In [13]:
print (len(pool.CropTyp.unique()))
print (len(GT_wMeta.CropTyp.unique()))

68
64


In [14]:
# pool = pool[pool.ExctAcr > 10].copy()
pool = pool[pool.CropTyp.isin(list(GT_wMeta.CropTyp.unique()))]

In [15]:
GT_wMeta.head(2)

,ID,Vote,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,99837_WSDA_SF_2017,2,barley,center pivot,wsda,73,73.250981,2017/10/10,Grant
1,114615_WSDA_SF_2017,1,barley,center pivot,wsda,31,30.993504,2017/10/10,Grant


In [16]:
numer = pd.DataFrame(GT_wMeta.groupby(["CropTyp"])["ID"].count()).reset_index()
numer.rename(columns={"ID": "numer"}, inplace=True)
numer.head(2)

,CropTyp,numer
0,alfalfa hay,43
1,alfalfa seed,23


In [17]:
denom = pd.DataFrame(pool.groupby(["CropTyp"])["ID"].count()).reset_index()
denom.rename(columns={"ID": "denom"}, inplace=True)
denom.head(2)

,CropTyp,denom
0,alfalfa hay,3343
1,alfalfa seed,337


In [18]:
in_df = pd.DataFrame({"CropTyp" : sorted(pool.CropTyp.unique())})
in_df.head(2)

,CropTyp
0,alfalfa hay
1,alfalfa seed


In [19]:
print (in_df.shape)
in_df = pd.merge(in_df, numer, on="CropTyp", how="outer")
print (in_df.shape)
in_df = pd.merge(in_df, denom, on="CropTyp", how="outer")
print (in_df.shape)
in_df.head(2)

(64, 1)
(64, 2)
(64, 3)


,CropTyp,numer,denom
0,alfalfa hay,43,3343
1,alfalfa seed,23,337


In [20]:
in_df["inclusion_prob"] = in_df["numer"] / in_df["denom"]

In [21]:
in_df.head(2)

,CropTyp,numer,denom,inclusion_prob
0,alfalfa hay,43,3343,0.012863
1,alfalfa seed,23,337,0.068249


In [22]:
in_df.head(2)

,CropTyp,numer,denom,inclusion_prob
0,alfalfa hay,43,3343,0.012863
1,alfalfa seed,23,337,0.068249


In [23]:
pool.head(2)

,ID,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Shp_Lng,Shap_Ar
0,51380_WSDA_SF_2016,Seed,bluegrass seed,6,sprinkler,2003/07/01 00:00:00,2016/09/19 00:00:00,wsda,NaN,T20R35E24,Adams,Wheat,6.116621,937.384083,24753.087193
1,51381_WSDA_SF_2016,Cereal Grain,wheat,117,center pivot,2003/07/01 00:00:00,2016/09/20 00:00:00,wsda,NaN,T16R36E22,Adams,Potato,116.743101,2445.413312,472442.568255


In [24]:
sorted(list(pool.Irrigtn.unique()))

['big gun',
 'big gun/center pivot',
 'big gun/sprinkler',
 'big gun/wheel line',
 'center pivot',
 'center pivot/drip',
 'center pivot/rill',
 'center pivot/sprinkler',
 'center pivot/sprinkler/wheel line',
 'center pivot/wheel line',
 'drip',
 'drip/big gun',
 'drip/micro-sprinkler',
 'drip/rill',
 'drip/sprinkler',
 'drip/wheel line',
 'flood',
 'hand',
 'micro-sprinkler',
 'rill',
 'rill/sprinkler',
 'rill/sprinkler/wheel line',
 'rill/wheel line',
 'sprinkler',
 'sprinkler/wheel line',
 'wheel line']

In [25]:
sorted(list(pool.DataSrc.unique()))

['conservation district', 'producer', 'wsda']

In [26]:
pool.county.unique()

array(['Adams', 'Benton', 'Franklin', 'Yakima', 'Grant', 'Walla Walla'],
      dtype=object)

In [27]:
pool["SF_year"] = pool["ID"].str.split("_", expand=True)[3]
pool["SF_year"] = pool["SF_year"].astype(int)

pool_Adams = pool[pool.county == "Adams"].copy()
pool_Benton = pool[pool.county == "Benton"].copy()
pool_Franklin = pool[pool.county == "Franklin"].copy()
pool_Yakima = pool[pool.county == "Yakima"].copy()
pool_Grant = pool[pool.county == "Grant"].copy()
pool_Walla = pool[pool.county == "Walla Walla"].copy()

In [28]:
print (pool_Adams.SF_year.unique())
print (pool_Benton.SF_year.unique())
print (pool_Franklin.SF_year.unique())
print (pool_Yakima.SF_year.unique())
print (pool_Grant.SF_year.unique())
print (pool_Walla.SF_year.unique())

[2016]
[2016]
[2018]
[2018]
[2017]
[2015]


In [29]:
in_df = in_df.round(3)

# out_name = "/Users/hn/Documents/01_research_data/Amin/inclusion.csv"
# in_df.to_csv(out_name, index = False)

In [73]:
in_df.head(2)

,CropTyp,numer,denom,inclusion_prob
0,alfalfa hay,43,3343,0.013
1,alfalfa seed,23,337,0.068


In [93]:
import pickle
from datetime import datetime
dir_ = "/Users/hn/Documents/01_research_data/NASA/Amin/"
filename = dir_ + "5_OverSampled_TestResults.sav"
five_OverSam_TestRes = pd.read_pickle(filename)
five_OverSam_TestRes.keys()

dict_keys(['ML_test_results', 'source_code', 'Author', 'Date'])

In [94]:
five_OverSam_TestRes = five_OverSam_TestRes["ML_test_results"]
five_OverSam_TestRes.keys()

dict_keys(['test_results_SVM', 'test_results_RF', 'test_results_KNN', 'test_results_DL'])

In [95]:
for _key in five_OverSam_TestRes.keys():
    # remove the 6th split. That's the one with different size
    if "train_ID6" in five_OverSam_TestRes[_key].keys():
        five_OverSam_TestRes[_key].pop("train_ID6")
    

In [97]:
for _key in five_OverSam_TestRes.keys():
    print (_key)
    print (five_OverSam_TestRes[_key].keys())

test_results_SVM
dict_keys(['train_ID1', 'train_ID2', 'train_ID3', 'train_ID4', 'train_ID5'])
test_results_RF
dict_keys(['train_ID1', 'train_ID2', 'train_ID3', 'train_ID4', 'train_ID5'])
test_results_KNN
dict_keys(['train_ID1', 'train_ID2', 'train_ID3', 'train_ID4', 'train_ID5'])
test_results_DL
dict_keys(['train_ID1', 'train_ID2', 'train_ID3', 'train_ID4', 'train_ID5'])


In [102]:
for _key in five_OverSam_TestRes.keys():
    for train_ID in five_OverSam_TestRes[_key].keys():
        five_OverSam_TestRes[_key][train_ID]["a_test_set_df"] = \
             pd.merge(five_OverSam_TestRes[_key][train_ID]["a_test_set_df"], 
                      pool[["ID", "CropTyp"]], how="left", on="ID")

In [110]:
five_OverSam_TestRes.keys()

dict_keys(['test_results_SVM', 'test_results_RF', 'test_results_KNN', 'test_results_DL'])

In [118]:
five_OverSam_TestRes['inclusion_prob'] = in_df

In [119]:
import pickle
from datetime import datetime
dir_ = "/Users/hn/Documents/01_research_data/NASA/Amin/"
filename = dir_ + "five_OverSam_TestRes_and_InclusionProb.sav"

export_ = {"five_OverSam_TestRes": five_OverSam_TestRes, 
           "source_code" : "inclusion.ipynb",
           "Author": "HN",
           "Date" : datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

pickle.dump(export_, open(filename, 'wb'))